In [1]:
import pandas as pd
import numpy as np

df = pd.read_pickle("../../data/interim/rating.pkl")
df

,user_id,item_id,parent_asin,rating,timestamp
0,0,548,B000CRFOMK,5.0,1422549495000
0,0,2528,B003MVZ60S,5.0,1340409020000
0,0,2671,B003XIJ566,5.0,1395513662000
0,0,2938,B004AM5RB6,1.0,1300396593000
0,0,3204,B004P15HD0,5.0,1374019952000
...,...,...,...,...,...
4998,4998,25141,B0BDT4L4FJ,4.0,1499699601275
4998,4998,25267,B0BGJLM3T7,4.0,1658051722244
4998,4998,26025,B0BS2SLDVY,4.0,1596555554593
4998,4998,26494,B0BYJSHNQ6,4.0,1636381963196


In [2]:
LEN_ITEM = len(df["item_id"].unique())

In [3]:
df = df.explode(["user_id", "rating", "timestamp"])
df

,user_id,item_id,parent_asin,rating,timestamp
0,0,548,B000CRFOMK,5.0,1422549495000
0,0,2528,B003MVZ60S,5.0,1340409020000
0,0,2671,B003XIJ566,5.0,1395513662000
0,0,2938,B004AM5RB6,1.0,1300396593000
0,0,3204,B004P15HD0,5.0,1374019952000
...,...,...,...,...,...
4998,4998,25141,B0BDT4L4FJ,4.0,1499699601275
4998,4998,25267,B0BGJLM3T7,4.0,1658051722244
4998,4998,26025,B0BS2SLDVY,4.0,1596555554593
4998,4998,26494,B0BYJSHNQ6,4.0,1636381963196


In [4]:
df = df.groupby("user_id").agg({
    "item_id": lambda x: x.tolist(),
    "rating": lambda x: x.tolist(),
}).reset_index(drop=False)

df["item_len"] = df["item_id"].apply(lambda x: len(x))

df

,user_id,item_id,rating,item_len
0,0,"[548, 2528, 2671, 2938, 3204, 6367, 7016, 7084...","[5.0, 5.0, 5.0, 1.0, 5.0, 1.0, 5.0, 5.0, 5.0, ...",56
1,1,"[117, 1100, 1292, 1432, 2116, 2525, 3373, 3876...","[2.0, 3.0, 5.0, 5.0, 5.0, 2.0, 2.0, 4.0, 5.0, ...",47
2,2,"[2625, 7017, 7516, 11056, 11439, 11582, 11713,...","[3.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, ...",62
3,3,"[12939, 14302, 17024, 17890, 19942, 21583, 216...","[5.0, 1.0, 5.0, 5.0, 5.0, 1.0, 1.0, 5.0, 5.0, ...",22
4,4,"[1863, 5147, 7084, 9860, 9871, 11527, 13686, 1...","[5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 3.0, 5.0, 4.0, ...",24
...,...,...,...,...
4994,4994,"[5103, 10367, 11104, 17703, 22934, 23785, 2424...","[5.0, 5.0, 5.0, 5.0, 4.0, 1.0, 4.0, 5.0, 5.0, ...",11
4995,4995,"[29, 597, 932, 1259, 2039, 2059, 2089, 2156, 3...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",49
4996,4996,"[78, 588, 701, 1106, 1250, 1379, 1506, 1875, 2...","[3.0, 3.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",93
4997,4997,"[260, 2364, 2370, 3518, 3589, 4679, 4906, 5468...","[5.0, 5.0, 5.0, 3.0, 5.0, 3.0, 5.0, 4.0, 5.0, ...",56


In [5]:
from tqdm import tqdm
import numpy as np
from copy import deepcopy

total_item_id = np.arange(LEN_ITEM)

for idx, row in tqdm(df.iterrows()):

    neg_item_idx = np.random.choice(total_item_id, size=len(row["item_id"]), replace=False)
    while(np.intersect1d(neg_item_idx, row["item_id"]).shape[0] > 0):
        neg_item_idx = np.random.choice(total_item_id, size=len(row["item_id"]), replace=False)
    # neg_item_idx = total_item_id[~np.isin(total_item_id, row["item_id"])]
    # neg_sampled_idx = np.random.choice([i for i in range(len(neg_item_idx))], size=len(row["item_id"]), replace=False)

    item_id = row["item_id"] + neg_item_idx.tolist()
    rating = row["rating"] + [0]*len(neg_item_idx)

    df.at[idx, "item_id"] = item_id
    df.at[idx, "rating"] = rating

df

4999it [00:03, 1587.14it/s]


,user_id,item_id,rating,item_len
0,0,"[548, 2528, 2671, 2938, 3204, 6367, 7016, 7084...","[5.0, 5.0, 5.0, 1.0, 5.0, 1.0, 5.0, 5.0, 5.0, ...",56
1,1,"[117, 1100, 1292, 1432, 2116, 2525, 3373, 3876...","[2.0, 3.0, 5.0, 5.0, 5.0, 2.0, 2.0, 4.0, 5.0, ...",47
2,2,"[2625, 7017, 7516, 11056, 11439, 11582, 11713,...","[3.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, ...",62
3,3,"[12939, 14302, 17024, 17890, 19942, 21583, 216...","[5.0, 1.0, 5.0, 5.0, 5.0, 1.0, 1.0, 5.0, 5.0, ...",22
4,4,"[1863, 5147, 7084, 9860, 9871, 11527, 13686, 1...","[5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 3.0, 5.0, 4.0, ...",24
...,...,...,...,...
4994,4994,"[5103, 10367, 11104, 17703, 22934, 23785, 2424...","[5.0, 5.0, 5.0, 5.0, 4.0, 1.0, 4.0, 5.0, 5.0, ...",11
4995,4995,"[29, 597, 932, 1259, 2039, 2059, 2089, 2156, 3...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",49
4996,4996,"[78, 588, 701, 1106, 1250, 1379, 1506, 1875, 2...","[3.0, 3.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",93
4997,4997,"[260, 2364, 2370, 3518, 3589, 4679, 4906, 5468...","[5.0, 5.0, 5.0, 3.0, 5.0, 3.0, 5.0, 4.0, 5.0, ...",56


In [6]:
df = df.explode(["item_id", "rating"])
df

,user_id,item_id,rating,item_len
0,0,548,5.0,56
0,0,2528,5.0,56
0,0,2671,5.0,56
0,0,2938,1.0,56
0,0,3204,5.0,56
...,...,...,...,...
4998,4998,345,0,35
4998,4998,12433,0,35
4998,4998,4898,0,35
4998,4998,3378,0,35


In [7]:
df.to_pickle("../../data/processed/rating_engage.pkl")

In [9]:
df["item_id"].max()

27582